In [47]:
#@title Import e funzioni utili
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from numba import njit
from scipy.optimize import curve_fit
import scipy.stats

In [48]:
#@title Clean data
#create a list of all the files in the folder
DIR = "../data/energia/LOB_ottobre21/LOB_ottobre21/"
lob_files =os.listdir(DIR)
lob_files.sort()
lst_order = []
for element in lob_files:
    # import data
    df = pd.read_csv(DIR+ element)
    df.fillna(0, inplace = True)

    # delete first two column and empty LOB
    df.drop(columns = ["Unnamed: 0", "key"], inplace = True)
    df.drop(df[df["AskPrice_0"] == 0].index.to_list(), inplace = True)
    df.drop(df[df["BidPrice_0"] == 0].index.to_list(), inplace = True)

    # scale price to dollar cent and add mid price and spread
    df["BidPrice_0"] = df["BidPrice_0"]
    df["AskPrice_0"] = df["AskPrice_0"]
    df["MidPrice"] = (df["BidPrice_0"] + df["AskPrice_0"]) / 2
    df["Spread"] = df["AskPrice_0"] - df["BidPrice_0"]

    # transform the column Datetime from string to datetime
    df["Datetime"]= pd.to_datetime(df["Datetime"])

    #create a new column that represent second to midnight 
    seconds = np.zeros(len(df))
    for i, date in enumerate(df["Datetime"]):
        seconds[i] = date.second + 60 * date.minute + 3600 * date.hour + \
                                    date.microsecond * 1e-6
    df["Time"] = seconds

    #delete rows with spread <= 0
    df = df.loc[df["Spread"] > 0]
    
    df = df.loc[df["Datetime"].dt.hour > 6]
    df = df.loc[df["Datetime"].dt.hour < 16]
    
    df = df.loc[df["Datetime"].dt.day != 27]
    
    lst_order.append(df)

clean_data = pd.concat(lst_order)
clean_data.reset_index(inplace = True, drop = True)

df = clean_data.iloc[:,1:41].diff().fillna(0)

In [63]:
#@title import trade data
#create a list of all the files in the folder
DIR_1 = "../data/energia/trade_ottobre2021_nuovo/trade_ottobre2021/"
trade_files =os.listdir(DIR_1)
trade_files.sort()
lst_trade = []
for element in trade_files:
    # import data
    df = pd.read_csv(DIR_1 + element)

    df.drop(columns = ["Unnamed: 0"], inplace = True)

    # transform the column Datetime from string to datetime
    df["DateTime"]= pd.to_datetime(df["DateTime"])

    seconds = np.zeros(len(df))
    for i, date in enumerate(df["DateTime"]):
      seconds[i] = date.second + 60 * date.minute + 3600 * date.hour + \
                  date.microsecond * 1e-6 + date.day * 3600 * 24

    df["Seconds"] = seconds

    df = df.loc[df["DateTime"].dt.hour > 6]
    df = df.loc[df["DateTime"].dt.hour < 16]
    
    df = df.loc[df["DateTime"].dt.day != 16]
    lst_trade.append(df)



trade_df = pd.concat(lst_trade)

In [66]:
DIR = "../data/energia/order/"

df = pd.read_csv(DIR + "order.csv", index_col=0)

In [67]:
a = df[(df["Price"]==0) & (df["Quote"] != "NoBest")].index.to_list()


In [68]:
for i in a:
    if df["Type"].at[i] == "Market/Cancel":
        q = df["Quote"].at[i]
        df["Type"].at[i] = "Limit"
        df["Price"].at[i] = clean_data["AskPrice_" + str(q)].at[i]
        df["Volume"].at[i] = clean_data["AskPrice_" + str(q)].at[i]
        
    elif df["Type"].at[i] == "Limit":
        q = df["Quote"].at[i]
        df["Type"].at[i] = "Market/Cancel"
        df["Price"].at[i] = clean_data["AskPrice_" + str(q)].at[i-1]
        df["Volume"].at[i] = clean_data["AskPrice_" + str(q)].at[i-1]
        

In [85]:
#@title Match: Tempo Segno Volume Prezzo (1)

df1 = df.copy()

lst_index = []
for i in range(len(trade_df)):
    p = trade_df["Price"].iat[i]
    v = trade_df["Volume"].iat[i]
    s = trade_df["Seconds"].iat[i]
    if trade_df["AggressorAction"].iat[i] == "Sell":
        sign = 1
    else:
        sign = -1

    k = 1
    a = []
    #verify if the broker is EEX in this case information on the sign is useless
    if trade_df["AggressorBroker"].iat[i] == "EEX":
        while a == [] and k < 5:
            a = df1[(df1["Volume"] == v) & (df1["Price"] == p) & (df1["Type"] == "Market/Cancel") \
                & (df1["Seconds"] > s - k) & (df1["Seconds"] < s + k)].index.to_list()
            k += 1
    
        if a == []:
            k = 1
            while a == [] and k < 5:
                a = df1[(df1["Volume"] == v) & (df1["Type"] == "Market/Cancel") \
                    & (df1["Seconds"] > s - k) & (df1["Seconds"] < s + k)].index.to_list()
                k += 1
            if a == []:
                k = 1
                while a == [] and k < 5:
                    a = df1[(df1["Type"] == "Market/Cancel") \
                        & (df1["Seconds"] > s - k) & (df1["Seconds"] < s + k)].index.to_list()
                    k += 1
    else:  
        while a == [] and k < 5:
            a = df1[(df1["Volume"] == v) & (df1["Price"] == p) & (df1["Type"] == "Market/Cancel") \
                & (df1["Seconds"] > s - k) & (df1["Seconds"] < s + k) & (df1["Sign"] == sign)].index.to_list()
            k += 1
            
        if a == []:
            k = 1
            while a == [] and k < 5:
                a = df1[(df1["Volume"] == v) & (df1["Price"] == p) & (df1["Type"] == "Market/Cancel") \
                    & (df1["Seconds"] > s - k) & (df1["Seconds"] < s + k)].index.to_list()
                k += 1

            if a == []:
                k = 1
                while a == [] and k < 5:
                    a = df1[(df1["Volume"] == v) & (df1["Type"] == "Market/Cancel") \
                        & (df1["Seconds"] > s - k) & (df1["Seconds"] < s + k)].index.to_list()
                    k += 1

                if a == []:
                    k = 1
                    while a == [] and k < 5:
                        a = df1[(df1["Type"] == "Market/Cancel") \
                            & (df1["Seconds"] > s - k) & (df1["Seconds"] < s + k)].index.to_list()
                        k += 1
    lst_index.append(a)

In [86]:
n1 = 0
for element in lst_index:
    if element == []:
        n1 +=1
n1

1536

In [88]:
df1["Broker"] = ["None" for i in range(len(df1))]
df1["BrokerSign"] = ["None" for i in range(len(df1))]
df1["PriceTrade"] = [0 for i in range(len(df1))]
df1["BrokerVolume"] = [0 for i in range(len(df1))]

for k,element in enumerate(lst_index):
  if element != []:
    # check if the random order was not preavously chosen
    flag = False
    # to avoid infinite loop repeat while at most 10 times
    i = 0
    while flag is False:
      trade = np.random.choice(element)
      if df1["Type"].at[trade] != "Market" or i>10:
        flag = True
      i += 1

    #check if trades was at the best quotes
    df1["Type"].at[trade] = "Market"
    # save broker, price and sign
    df1["Broker"].at[trade] = trade_df["AggressorBroker"].iat[k]
    df1["BrokerSign"].at[trade] =  trade_df["AggressorAction"].iat[k]
    df1["PriceTrade"].at[trade] =  trade_df["Price"].iat[k]
    df1["BrokerVolume"].at[trade] =  trade_df["Volume"].iat[k]

# all remaining market/cancel are cancel order
df1["Type"].replace(["Market/Cancel"], "Cancel", inplace = True)

In [91]:
df1.to_csv("../data/best_match.csv")

In [93]:
df1["DateTime"] = pd.to_datetime(df1["DateTime"])

In [96]:
#@title Stima parametri (ignora non matching orders)
days = pd.unique(clean_data["Datetime"].dt.day)

lenght = len(days)
l = []
u = []
delta = []
date = []
spread = []
volatility = []

for i,element in enumerate(days):
    # see what happens at the best quotes
    X_lo = df1.loc[(df1["Type"] == "Limit") & (df1["Quote"] == "0") & (df1["DateTime"].dt.day == element)]
    X_mo = df1.loc[df1["Type"] == "Market"].loc[df1["DateTime"].dt.day == element]
    X_c = df1.loc[(df1["Type"] == "Cancel") & (df1["Quote"] == "0") & (df1["DateTime"].dt.day == element)]

    df = clean_data.loc[clean_data["Datetime"].dt.day == element]

    N_lo = len(X_lo)
    N_mo = len(X_mo)
    N_c = len(X_c)

    tot = N_lo + N_mo + N_c

    X_lo["Spread"] =  df["Spread"]*100

    #compute lambda, mu, delta
    v0 = X_lo["Volume"].mean()
    vv = (df["BidVolume_0"].mean() + df["AskVolume_0"].mean()) / 2
    u.append(0.5 / tot / v0 * (X_mo["Volume"].sum()))
    all_l = .5 * N_lo / tot
    n = 2 * (1 + ((X_lo["Spread"] // 2).mean()))
    l.append(all_l / n)
    delta.append(.5 / tot / vv * (X_c["Volume"].sum()))
    #compute volatility and mid spread for the day
    date.append(element)
    spread.append(df["Spread"].multiply(100).mean())
    mp = np.log(df["MidPrice"].multiply(100).to_numpy())
    volatility.append(np.sqrt(((mp[1:]- mp[:-1])**2).mean()))

str_date = [str(int(day)) + "/10" for day in date]

params = pd.DataFrame(str_date,columns=["Date"])
params["Lambda"] = l
params["Mu"] = u
params["Delta"] = delta
params["MeanSpread"] = spread
params["Volatility"] = volatility

<ipython-input-96-9c2314c6abdd>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_lo["Spread"] =  df["Spread"]*100


In [99]:
params.to_csv("../data/params_ZI.csv")